# Basic design of RF systems

S. Albright, H. Damerau, A. Lasheen, F. Tecker, C. Völlinger

## Links

- Introductory CAS wesite: https://indico.cern.ch/event/1117526/
- Programme of the CAS: https://cas.web.cern.ch/sites/default/files/Timetable_Introductory2022_ver7.pdf
- Python software installation for transverse (and longitudinal) exercises: https://github.com/cerncas/hands-on-lattice-exercises/blob/master/Setup_Instructions.md
- Longitudinal hands-on, link to content and cheat sheets: https://indico.cern.ch/event/1117526/contributions/4978478/

# Introduction

In this hands-on session we will either
- design the RF system of a proton synchrotron, the superconduding SPS (scSPS)
- or develop the RF system for a hypothetical beam energy and current upgrade of an electron storage ring (Soleil).

**Please make your choice according to your interest.**

## Useful formula for bucket area reduction factor depending on stable phase
- Bucket area reduction ratio depdening on stable phase
- Use approximation (S. Y. Lee book, p. 242): $\alpha(\phi_\mathrm{S}) \simeq \cfrac{1 - \sin \phi_\mathrm{S}}{1 + \sin \phi_\mathrm{S}}$

## Import modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as sciCont

# Upgrade the CERN-SPS to higher energy as an injector for future circular colliders

### Design an RF system optimized for the Superconducting Super Proton Synchrotron (scSPS)
(see, e.g. <a href="https://indico.cern.ch/event/591312/contributions/2386529/attachments/1379193/2133450/scSPS_Note_v2.pdf">F. Burkart et al., SPS energy upgrade considerations</a>)

In this hands-on session we will design the RF system for a proton synchrotron.

The goal of the session is to calculate the relevant longitudinal parameters.

The notebook is constructed with the following purpose in mind:

1. Design a linear acceleration cycle in bending field which determines average energy gain per turn and stable phase.
2. Knowing energy gain per turn and the number of particles accelerated, the average RF power to the beam is calculated.
3. Injection and extraction energies also fix the revolution frequencies and, in combination with the harmonic number, define the the RF frequency range.
4. The acceleration of a bunch with a given size in the longitudinal phase space, i.e. longitudinal emttance, requires a sufficient bucket area. This area is defined by the RF voltage. Interesting effects occur around the energy of transiton crossing, which also deserves a more detailed look.
5. With the knowledge of frequency and voltage the fundamental reqirements are set. Look for existing RF systems in particle accelerators and suggest your design. No calculations are needed for these parameter estimates. However please justify your choice(s).
6. The energy transfer does not only take place from the cavity to the beam, but also the beam may induce significant voltage back into the cavity. This phenomenon has been introduced as beam loading.
7. New RF systems for particle accelerators are rarely designed from scratch, but inspired by existing installations. Compare your RF system design with the existing one of the CERN SPS, as well as with the RF system of the now-decommissioned Tevatron, or any other RF system you may find.

**Basic parameters of the superconducting Proton Synchrotron (scSPS) at CERN**

| Parameter                        |                                                                     |
| -------------------------------- | ------------------------------------------------------------------- |
| Energy range                     | $E_\mathrm{kin} = 13.1\,\mathrm{GeV}\ldots1300\,\mathrm{GeV}$       |
| Circumference                    | $2 \pi R = 6911.5\,\mathrm{m}$                                      |
| Bending radius                   | $\rho = 741.3\,\mathrm{m}$                                          |
| Transition gamma                 | $\gamma_\mathrm{tr} = 18$                                           |
| Acceleration time                | $4\,\mathrm{s}$                                                     |
| Longitudinal emittance per bunch | $\varepsilon_\mathrm{l} = 0.4\,\mathrm{eVs}\ldots0.5\,\mathrm{eVs}$ |
| Maximum bucket filling factor    | $\varepsilon_\mathrm{l}/A_\mathrm{bucket} = 0.8$                    |
| Total beam intensity             | $N = 1 \cdot 10^{13} \,\mathrm{protons}$                            |
| Minimum bunch spacing            | $25 \,\mathrm{ns}$                                                  |

## Exercise 1: Average energy gain and stable phase

- How much energy does the particle gain during each turn assuming a constant ramp rate in bending field, $B$?
    - Assume a linear acceleration ramp $dB/dt = \mathrm{const.}$
- What would the stable phase be for an RF voltage of 20 MV? 

### Answer:
$$
\Delta E_{\text{turn}} = 2\pi q\rho R\dot B = (2\pi R)\times \rho \times q\frac{dB}{dt}
$$

$$
R_{\text{mag. rig.}} = B\rho = \frac{pc}q
$$

$$
p/q = B\rho
$$

KE -> Momentum

$$
T_E = E_p + E_k = + m_0c^2
$$

$$
1300^2 = (pc)^2 + E_0^2 = (pc)^2 + (m_0c^2)^2
$$

$$
\therefore p = \sqrt{{E_T^2 - m_0^2}}


In [34]:
CIRCUMFERENCE = 6911.5
RADIUS = 741.3

ENERGY_GAIN = 1300E9 - 13.1E9
RAMP_TIME = 4

PROTON_MASS = 938E6

INJ_MOMENTUM = np.sqrt( (13.1E9 + PROTON_MASS)**2 - PROTON_MASS**2)

EXT_MOMENTUM = np.sqrt( (1300E9 + PROTON_MASS)**2 - PROTON_MASS**2)

INJ_B = (INJ_MOMENTUM / (RADIUS * 1)) / 3E8

EXT_B = (EXT_MOMENTUM / (RADIUS * 1)) / 3E8

DELTA_B = EXT_B - INJ_B

DELTA_ENERGY_PER_TURN = CIRCUMFERENCE * RADIUS * (DELTA_B / RAMP_TIME)

print(f'Delta E per turn = {DELTA_ENERGY_PER_TURN} eV')

Delta E per turn = 7412186.539410733 eV


## Exercise 2: Power transfer to the beam

- How much power is transferred from the RF system to the beam?

### Answer

$$
P_\text{transferred} = \text{Energy per turn} \times \text{Time per turn} \times \text{Number turns}
$$

In [41]:
ENERGY_CHANGE_PER_TIME = ENERGY_GAIN / RAMP_TIME
N_PROTONS = 1E13

POWER_eV = ENERGY_CHANGE_PER_TIME * N_PROTONS
POWER_J = POWER_eV * 1.6E-19

print(f'Power transferred = DeltaE / T * Nprotons\t=\t{POWER_J} J/s')
print(f"\t\t\t\t\t\t\t\t\t\t{POWER_J*1e-6} MJ/s")

Power transferred = DeltaE / T * Nprotons	=	514760.0 J/s
										0.51476 MJ/s


## Exercise 3: RF frequency and harmonic

- Choose RF frequency and harmonic of the RF system.
- Note a few arguments for your choice.
- What is the frequency range of the RF system?

25 ns bunch spacing
at least 1/25ns lower bound

and 11 and 20 and 6

so a multiple of 52.8

Pick 5.28 MHz - not super high frequency requiring huge gradients, but high enough to be efficient

## Exercise 4: Calculate bucket area during the cycle, determine RF voltage along the cycle

- Plot momentum, kinetic energy and revolution frequency (and/or further parameters) during the cycle.
- Calculate and plot the bucket area along the cycle and choose an RF voltage such that a bunch with $0.45\,\mathrm{eVs}$ longitudinal emittance can be comfortably accelerated, e.g $\varepsilon_\mathrm{l}/A_\mathrm{bucket} \simeq 0.8$.
- Zoom around transition crossing. What happens there?

#### Bucket area reduction factor

In [2]:
def reduction_ratio(deltaE, voltage):    
    phis = np.arcsin(deltaE/voltage)
    return (1-np.sin(phis))/(1+np.sin(phis))

## Exercise 5: RF cavity, number of cavities, RF amplifier power

- Choose an approriate type of RF cavity.
- How many cavities would you install?
- Please note some arguments for the discussion.
- Also have a look at the Tevatron RF system.

## Exercise 6: Requirements for beam loading

- What is the beam induced voltage and power due to the passage of one bunch?
- Under which circumstances do you really need that power?

- This additional power would be needed to fully compensate beam loading and operate the cavity at any phase

## Exercise 7: Comparison with RF systems present SPS and at Fermilab Tevatron

- Compare the parameters of your RF system with the ones of the present SPS and the Tevatron.

# Design of an RF system upgrade for an electron storage ring

### Design an RF system to run the Soleil electron storage ringe at higher energy and beam current

In this hands-on session we will develop the RF system for a hypothetical beam energy and current upgrade for an electron storage ring (Soleil). As a storage ring the particle energy should just be kept constant.

The goal of the session is to calculate the relevant longitudinal parameters.

The notebook is constructed with the following purpose in mind:

1. Start from the dominating energy loss per turn due to synchrotron radiation.
2. Knowing energy loss per turn and the number of particles stored the average RF power to the beam is calculated.
3. The RF frequency can be any integer multiple of the revolution frequency. Collect arguments for the choice of the RF frequency.
4. With the knowledge of frequency and voltage the fundamental reqirements are set. Look for existing RF systems in particle accelerators and suggest your design. No calculations are needed for these parameter estimates. However please justify your choice(s).
5. The energy transfer does not only take place from the cavity to the beam, but also the beam may induce significant voltage back into the cavity. This phenomenon has been introduced as beam loading.
6. Without the RF system, the beam quickly loses energy. It is interesting to estimate how many turns it would survive and how rapidly its energy decreases.
7. Synchrotron radiation has the advantage of introducing damping and any oscillations of particles  reduced. The characteristic timescale of that process is the synchrotron radiation damping time.
8. New RF systems for particle accelerators are rarely designed from scratch, but inspired by existing installations. Compare your RF system design with the existing one of Soleil, as well as with the RF system of the larger ESRF main ring.

Key upgrade parameters:
- Higher energy: $2.75\,\mathrm{GeV}$ instead of ($3.5\,\mathrm{GeV}$).
- Higher beam current: $800\,\mathrm{mA}$ instead of $500\,\mathrm{mA}$.
- Bunch spacing of $25\,\mathrm{ns}$.
- Design the new RF system which can work in combination with the existing one.

**Basic parameters of the Soleil electron storage ring ([parameter table](https://www.synchrotron-soleil.fr/en/research/sources-and-accelerators/parameters-accelerators-storage-ring))**

| Parameter             |                                                                   |
| --------------------- | ----------------------------------------------------------------- |
| Beam energy           | $E = 2.75\,\mathrm{GeV}\rightarrow 3.5\,\mathrm{GeV}$             |
| Beam current          | $I_\mathrm{b} = 500\,\mathrm{mA} \rightarrow 800\,\mathrm{mA}$    |
| Circumference         | $2 \pi R = 354.097\,\mathrm{m}$                                   |
| Bending radius        | $\rho = 5.36\,\mathrm{m}$                                         |
| Phase slip factor     | $\eta = 1/\gamma^2_\mathrm{tr} - 1/\gamma^2 = 4.16 \cdot 10^{-4}$ |
| Harmonic of RF system | $h = 416$                                                         |
| RF frequency          | $f_\mathrm{RF} = 352.2\,\mathrm{MHz}$                             |

## Exercise 1: Average energy loss per turn

- Calculate the average energy loss per turn to be restituted before and after the upgrade.
- Plot the energy loss versus beam energy.

## Exercise 2: Average RF power

- What is the average power to the beam before and after the upgrade?
- Plot the required RF power versus beam energy.
- Why should the installed RF power actually be higher?

## Exercise 3: Chose RF frequency. Arguments?

- Choose the RF frequency and harmonic of the additional RF system.
- Note a few arguments supporting your choice.

- Must be integer harmonic of existing RF system ($h=416$)
- Chose twice that frequency ($h=832$, $f_\mathrm{RF} = 704$ MHz) to generate additional voltage more easily and with compact cavities

## Exercise 4: RF cavity, number of cavities, RF amplifier power

- Choose an approriate type of RF cavity.
- How many cavities would you install?
- Please note some arguments for the discussion.

## Exercise 5: Requirements for beam loading: beam induced voltage and power

- Calculate the beam induced voltage and power in the additional cavity.
- How does the power compare to the power lost by synchrotron radiation?
- Under which circumstances do you really need that power?

## Exercise 6: Beam life time with no RF

- How many turns would the beam survive without RF? For a first estimate one can assume a constant energy loss. The momentum acceptance is on the order of 0.5%.
- Optionally: take into account the energy loss per turn changes with beam energy.
- Plot the beam energy versus number of turns.

## Exercise 7: Radiation damping time

- Calculate the damping times of the synchrotron oscillations before and after the upgrade.

## Exercise 8: Comparison with RF system at ESRF

- Compare the parameters of the (additional) RF system with the one of the storage ring at ESRF.